In [20]:
import tkinter as tk
from tkinter import filedialog, messagebox, MULTIPLE, StringVar, OptionMenu
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os

class DataProcessingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("数据处理工具")
        self.root.geometry("1000x600")  # 设置窗口大小

        # 创建并排放置的框架
        self.button_frame = tk.Frame(root)
        self.button_frame.pack(pady=20)

        # 读取Excel数据按钮
        self.load_excel_button = tk.Button(self.button_frame, text="读取Excel数据", command=self.load_excel_data)
        self.load_excel_button.grid(row=0, column=0, padx=10)

        # 读取CSV数据按钮
        self.load_csv_button = tk.Button(self.button_frame, text="读取CSV数据", command=self.load_csv_data)
        self.load_csv_button.grid(row=0, column=1, padx=10)

        # 处理数据按钮
        self.process_button = tk.Button(self.button_frame, text="处理数据", command=self.process_data)
        self.process_button.grid(row=0, column=2, padx=10)

        # 保存数据按钮
        self.save_button = tk.Button(self.button_frame, text="保存相关系数矩阵", command=self.save_data)
        self.save_button.grid(row=0, column=3, padx=10)

        # 关闭按钮
        self.quit_button = tk.Button(self.button_frame, text="关闭", command=self.quit_application)
        self.quit_button.grid(row=0, column=4, padx=10)

        # 创建用于显示列名的框架
        self.columns_frame = tk.Frame(root)
        self.columns_frame.pack(pady=10)

        # 列名列表框
        self.columns_listbox = tk.Listbox(root, selectmode=MULTIPLE, width=80)
        self.columns_listbox.pack(pady=10)

        # 删除列按钮
        self.delete_columns_button = tk.Button(root, text="删除选中列", command=self.delete_selected_columns)
        self.delete_columns_button.pack(pady=10)

        # 文件格式选择下拉菜单
        self.file_format = StringVar(root)
        self.file_format.set("选择保存格式")  # 设置默认值
        self.format_menu = OptionMenu(root, self.file_format, "Excel (.xlsx)", "CSV (.csv)")
        self.format_menu.pack(pady=10)

        # 相关性计算方法选择下拉菜单
        self.correlation_method = StringVar(root)
        self.correlation_method.set("选择相关性计算方法")  # 设置默认值
        self.correlation_menu = OptionMenu(root, self.correlation_method, "皮尔逊", "斯皮尔曼", "肯德尔")
        self.correlation_menu.pack(pady=10)

        # 存储 DataFrame 和相关系数矩阵
        self.df = None
        self.df_scaled = None
        self.correlation_matrix = None
        self.last_file_name = None  # 存储上次读取的文件名

    def update_columns_display(self):
        # 清除旧的列名标签
        for widget in self.columns_frame.winfo_children():
            widget.destroy()

        if self.df is not None:
            # 逐行排列列名，每行最多显示10个列名
            for idx, col in enumerate(self.df.columns):
                label = tk.Label(self.columns_frame, text=col)
                label.grid(row=idx // 10, column=idx % 10, padx=5, pady=5)
            
            # 更新列表框中的列名
            self.columns_listbox.delete(0, tk.END)
            for col in self.df.columns:
                self.columns_listbox.insert(tk.END, col)
        else:
            # 如果没有加载任何数据，清空列表框
            self.columns_listbox.delete(0, tk.END)

    def load_excel_data(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if not file_path:
            return
        
        try:
            self.df = pd.read_excel(file_path)
            self.update_columns_display()
            self.last_file_name = os.path.splitext(os.path.basename(file_path))[0]  # 提取文件名，不带扩展名
            messagebox.showinfo("成功", "Excel数据已成功读取！")
        except Exception as e:
            messagebox.showerror("错误", str(e))

    def load_csv_data(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if not file_path:
            return
        
        try:
            self.df = pd.read_csv(file_path)
            self.update_columns_display()
            self.last_file_name = os.path.splitext(os.path.basename(file_path))[0]  # 提取文件名，不带扩展名
            messagebox.showinfo("成功", "CSV数据已成功读取！")
        except Exception as e:
            messagebox.showerror("错误", str(e))

    def delete_selected_columns(self):
        selected_indices = self.columns_listbox.curselection()
        selected_columns = [self.columns_listbox.get(i) for i in selected_indices]

        if not selected_columns:
            messagebox.showwarning("警告", "请先选择要删除的列！")
            return

        try:
            self.df = self.df.drop(columns=selected_columns)
            self.update_columns_display()
            messagebox.showinfo("成功", f"已删除选中的列：{', '.join(selected_columns)}")
        except Exception as e:
            messagebox.showerror("错误", str(e))

    def process_data(self):
        if self.df is None:
            messagebox.showwarning("警告", "请先读取数据！")
            return

        method = self.correlation_method.get()
        if method == "选择相关性计算方法":
            messagebox.showwarning("警告", "请先选择相关性计算方法！")
            return

        try:
            scaler = StandardScaler()
            scaled_data = scaler.fit_transform(self.df)
            self.df_scaled = pd.DataFrame(scaled_data, columns=self.df.columns)

            # 选择相关性计算方法
            if method == "皮尔逊":
                self.correlation_matrix = self.df_scaled.corr(method='pearson')
            elif method == "斯皮尔曼":
                self.correlation_matrix = self.df_scaled.corr(method='spearman')
            elif method == "肯德尔":
                self.correlation_matrix = self.df_scaled.corr(method='kendall')

            messagebox.showinfo("成功", "数据已成功处理！")
        except Exception as e:
            messagebox.showerror("错误", str(e))

    def save_data(self):
        if self.correlation_matrix is None:
            messagebox.showwarning("警告", "请先处理数据！")
            return
        
        file_format = self.file_format.get()
        if file_format == "选择保存格式":
            messagebox.showwarning("警告", "请先选择保存格式！")
            return

        if self.last_file_name is None:
            messagebox.showwarning("警告", "没有读取任何文件，无法生成默认文件名！")
            return

        try:
            # 根据相关性计算方法生成文件名
            method = self.correlation_method.get()
            if method == "选择相关性计算方法":
                method = "无相关性计算方法"

            default_file_name = f"{self.last_file_name}_{method}_相关系数矩阵"
            
            if file_format == "Excel (.xlsx)":
                # 保存相关系数矩阵为Excel
                output_file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", initialfile=default_file_name + ".xlsx", filetypes=[("Excel files", "*.xlsx")])
                if output_file_path:
                    self.correlation_matrix.to_excel(output_file_path)
            
            elif file_format == "CSV (.csv)":
                # 保存相关系数矩阵为CSV
                output_file_path = filedialog.asksaveasfilename(defaultextension=".csv", initialfile=default_file_name + ".csv", filetypes=[("CSV files", "*.csv")])
                if output_file_path:
                    self.correlation_matrix.to_csv(output_file_path)
            
            messagebox.showinfo("成功", f"相关系数矩阵已保存至 {output_file_path}")
        except Exception as e:
            messagebox.showerror("错误", str(e))

    def quit_application(self):
        self.root.destroy()  # 销毁所有窗口并结束程序

# 创建主窗口并运行应用
root = tk.Tk()
app = DataProcessingApp(root)
root.mainloop()
